In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from uuid import uuid4
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from rag import textbookRAG

In [2]:
from test_metadata import metadata


In [3]:
book_dir = Path("../utilities/book")

In [3]:
rag = textbookRAG(metadata=metadata)
rag.extract_from_metadata()

/Users/bryanmelvison/Code/FYP/KGV-AI-Tutor/backend/app/utilities/rag.py:55: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(


False

In [6]:
rag

In [8]:
a = rag.search("Can you tell me more about photosynthesis?", subject = "Biology", chapter = 1)

In [9]:
print(a)

- **CHARACTERISTICS OF PLANTS:**
- MULTICELLULAR STRUCTURE, WITH CELLS CONTAINING CHLOROPLASTS FOR PHOTOSYNTHESIS.
- PHOTOSYNTHESIS CONVERTS WATER AND CARBON DIOXIDE INTO ORGANIC COMPOUNDS LIKE STARCH AND SUCROSE.

#### FIGURE 2.1
- (A) A PEA PLANT: LEAVES AND STEM CELLS CONTAIN CHLOROPLASTS, GIVING THEM THEIR GREEN COLOR. WHITE FLOWERS ARE POLLINATED BY INSECTS.
- (B) MAIZE PLANTS ARE POLLINATED BY WIND; MALE FLOWERS PRODUCE POLLEN.
- (C) FEMALE MAIZE FLOWERS PRODUCE SEEDS AFTER POLLINATION.

THE STRUCTURE AND FUNCTION OF FLOWERING PLANTS WILL BE EXPLORED IN UNIT 3.

## CONCLUSION
UNDERSTANDING ORGANISM CLASSIFICATION APPRECIATES LIFE'S COMPLEXITY. EACH GROUP PLAYS A ROLE IN ECOSYSTEMS, AIDING EFFECTIVE STUDY THROUGH RECOGNITION OF THEIR FEATURES.
### CHLOROPLASTS

CHLOROPLASTS ARE UNIQUE TO PLANT CELLS AND ARE ESSENTIAL FOR PHOTOSYNTHESIS. THESE ORGANELLES CONTAIN CHLOROPHYLL, THE PIGMENT NECESSARY FOR ABSORBING LIGHT ENERGY, WHICH IS THEN CONVERTED INTO CHEMICAL ENERGY THROUGH PHOTO

In [7]:
rag.get_chapters_for_subject("Biology")

{1: 'Life Processes',
 2: 'The Variety of Living Organisms',
 3: 'Breathing and Gas Exchange',
 4: 'Food and Digestion',
 5: 'Blood and Circulation',
 6: 'Coordination',
 7: 'Chemical Coordination',
 8: 'Homeostasis and Excretion',
 9: 'Reproduction in Humans',
 10: 'Plants and Food',
 11: 'Transport in Plants',
 12: 'Chemical Coordination in Plants',
 13: 'Reproduction in Plants',
 14: 'Ecosystems',
 15: 'Human Influences on the Environment',
 16: 'Chromosomes, Genes and DNA',
 17: 'Cell Division',
 18: 'Genes and Inheritance',
 19: 'Natural Selection, Evolution and Selective Breeding',
 20: 'Using Microorganisms',
 21: 'Genetic Modification'}

In [8]:
rag.get_all_subjects()

['Biology']